In [ ]:
pip install xgboost -i https://pypi.tuna.tsinghua.edu.cn/simple

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
detail = pd.read_csv("details.csv")

In [49]:
# 创建列名映射字典
rename_dict = {
    '城市': '城市',
    '板块': '板块',
    '环线位置': '环线',
    '小区地址': '交通出行',
    '物业类别': '房屋用途',
    '建筑结构': '建筑结构',
    '产权描述': '产权所属',
    'coord_x': 'lon',
    'coord_y': 'lat'
}

# 重命名列
detail.rename(columns=rename_dict, inplace=True)

#删除无用信息
detail.drop(columns=['物业办公电话'], inplace=True)

In [50]:
train.drop(columns=['抵押信息'], inplace=True)

In [51]:
# 包含缺失值的列
missing_columns = train.columns[train.isnull().any()]
train[missing_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102117 entries, 0 to 102116
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   环线      39614 non-null   object
 1   房屋户型    101382 non-null  object
 2   套内面积    22331 non-null   object
 3   建筑结构    101382 non-null  object
 4   装修情况    101382 non-null  object
 5   梯户比例    99876 non-null   object
 6   配备电梯    90451 non-null   object
 7   别墅类型    1048 non-null    object
 8   上次交易    63850 non-null   object
 9   房屋年限    62879 non-null   object
 10  房屋优势    80306 non-null   object
 11  核心卖点    84127 non-null   object
 12  户型介绍    20823 non-null   object
 13  周边配套    64451 non-null   object
 14  交通出行    64876 non-null   object
dtypes: object(15)
memory usage: 11.7+ MB


In [52]:
# 不含缺失值的列
full_columns = train.columns[~train.isnull().any()]
train[full_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102117 entries, 0 to 102116
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   城市      102117 non-null  object 
 1   区域      102117 non-null  float64
 2   板块      102117 non-null  float64
 3   小区名称    102117 non-null  object 
 4   价格      102117 non-null  int64  
 5   所在楼层    102117 non-null  object 
 6   建筑面积    102117 non-null  object 
 7   房屋朝向    102117 non-null  object 
 8   交易时间    102117 non-null  object 
 9   交易权属    102117 non-null  object 
 10  房屋用途    102117 non-null  object 
 11  产权所属    102117 non-null  object 
 12  lon     102117 non-null  float64
 13  lat     102117 non-null  float64
 14  年份      102117 non-null  int64  
dtypes: float64(4), int64(2), object(9)
memory usage: 11.7+ MB


In [53]:
def fill_inside_space(df):
    X = train_data[['建筑面积']]
    y = train_data['套内面积']

    poly = PolynomialFeatures(degree=2)  # 选择2阶多项式
    X_poly = poly.fit_transform(X)

    model = LinearRegression()
    model.fit(X_poly, y)

    # 预测缺失值
    missing_data = df[df['套内面积'].isnull()]
    X_missing = poly.transform(missing_data[['建筑面积']])
    predicted_values = model.predict(X_missing)

    # 填充缺失值
    df.loc[df['套内面积'].isnull(), '套内面积'] = predicted_values

In [54]:
def data_process(train, test, detail, stage):
    df_detail = detail.copy()
    
    if stage == 0:
        df = train
    else:
        df = test
    # 1.填充缺失值
    
    # (1)建筑结构
    # 用 小区名称&建筑结构 填充 建筑结构
    for group_name, group in df.groupby('小区名称'):
        # 获取该小区的已知建筑结构
        known_structure = group.loc[group['建筑结构'].notnull() & (group['建筑结构'] != '未知结构'), '建筑结构']
        # 如果该小区有已知建筑结构，则填充缺失值和 '未知结构其他
        if not known_structure.empty:
            df.loc[df['小区名称'] == group_name, '建筑结构'] = df.loc[df['小区名称'] == group_name, '建筑结构'].replace({'未知结构': known_structure.iloc[0], None: known_structure.iloc[0]})
    df['建筑结构'].fillna('未知结构', inplace=True) # 剩下空值填'无'
    
    # (2)装修情况
    # 用 小区名称&装修情况 填充 建装修情况
    for group_name, group in df.groupby('小区名称'):
        # 获取该小区的已知装修情况
        known_structure = group.loc[group['装修情况'].notnull() & (group['装修情况'] != '其他'), '装修情况']
        # 如果该小区有已知装修情况，则填充缺失值和 '未知结构'
        if not known_structure.empty:
            df.loc[df['小区名称'] == group_name, '装修情况'] = df.loc[df['小区名称'] == group_name, '装修情况'].replace({'其他': known_structure.iloc[0], None: known_structure.iloc[0]})
    df['装修情况'].fillna('其他', inplace=True) # 剩下空值填'无'
    
    # (3)配备电梯
    df.loc[df['梯户比例'].notnull() & df['配备电梯'].isnull(), '配备电梯'] = '有' #用 梯户比例 填充 配备电梯
    df.loc[df['别墅类型'].notnull() & df['配备电梯'].isnull(), '配备电梯'] = '无' #用 别墅类型 填充 配备电梯
    df.loc[df['所在楼层'].str.contains('地下室', na=False), '配备电梯'] = '无'    #用 所在楼层 填充 配备电梯
    for group_name, group in df.groupby('小区名称'):                            #用 小区名称&配备电梯 填充 配备电梯
        if '有' in group['配备电梯'].values:  # 如果该小区存在 '有' 电梯的记录
            df.loc[df['小区名称'] == group_name, '配备电梯'] = '有'
        else:  # 如果该小区没有 '有' 电梯的记录
            df.loc[df['小区名称'] == group_name, '配备电梯'] = '无'
    df['配备电梯'].fillna('无', inplace=True) # 剩下空值填'无'
    
    # (4)房屋户型
    df['房屋户型'].fillna('1室0厅0厨0卫', inplace=True) # 剩下空值填'1室0厅0厨0卫',因为房屋面积都极小
    df['室'] = df['房屋户型'].str.extract(r'(\d+)室').astype(float)
    df['厅'] = df['房屋户型'].str.extract(r'(\d+)厅').astype(float)
    df['厨'] = df['房屋户型'].str.extract(r'(\d+)厨').astype(float)
    df['卫'] = df['房屋户型'].str.extract(r'(\d+)卫').astype(float)
    
    # (5)套内面积
    #去除单位
    df['建筑面积'] = df['建筑面积'].str.replace(r'[^\d\.]', '', regex=True).astype(float)
    df['套内面积'] = df['套内面积'].str.replace(r'[^\d\.]', '', regex=True).astype(float)
    
    # (6)梯户比例
    chinese_num_map = {
        '一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, '十': 10,
        '零': 0, '百': 100, '千': 1000, '万': 10000
        }

    #文本转数字
    def chinese_to_num(chinese_str):
        if pd.isna(chinese_str):  # 如果为空值，返回0
            return 0
            
        total = 0
        unit = 1 # 纪录当前位数
        for char in reversed(chinese_str):
            if char in chinese_num_map:
                digit = chinese_num_map[char]
                if digit >= 10:
                    if unit > 1:
                        total += unit
                    unit = digit
                else:
                    total += digit * unit
            else:
                total += unit
                unit = 1
        return total

    df['梯数'] = df['梯户比例'].str.extract(r'(\S+)梯')[0].apply(chinese_to_num)
    df['户数'] = df['梯户比例'].str.extract(r'(\S+)户')[0].apply(chinese_to_num)
    df['梯户比值'] = df['户数']/df['梯数']
    df['梯户比值'] = df['梯户比值'].replace([np.nan, np.inf], 0)
    
    # 2.转化为目标格式
    
    # (1)类别型变量
    # categorical_cols=['城市', '建筑结构', '装修情况', '配备电梯', '房屋年限', '产权所属', '室', '厅', '厨', '卫']

    # # LabelEncoder
    # label_encoders={}
    # for col in categorical_cols:
    #     le=LabelEncoder()
    #     df[col]=le.fit_transform(df[col])
    #     label_encoders[col]=le
        
    #(2)分离出数值型变量
    df['avg_lon'] = df.groupby('城市')['lon'].transform('mean')
    df['avg_lat'] = df.groupby('城市')['lat'].transform('mean')
    # 将纬度和经度从度转换为弧度
    df['lat_rad'] = np.radians(df['lat'])
    df['lon_rad'] = np.radians(df['lon'])
    df['avg_lat_rad'] = np.radians(df['avg_lat'])
    df['avg_lon_rad'] = np.radians(df['avg_lon'])

    # 计算两点之间的球面距离
    R = 6371  # 地球半径，单位：千米
    df['distance'] = 2 * R * np.arcsin(np.sqrt(
        np.sin((df['avg_lat_rad'] - df['lat_rad']) / 2) ** 2 +
        np.cos(df['lat_rad']) * np.cos(df['avg_lat_rad']) *
        np.sin((df['avg_lon_rad'] - df['lon_rad']) / 2) ** 2
))
    
    # 价格取对数
    if stage==0:
        df['log_prices'] = np.log(df['价格'])


In [ ]:
data_process(train, test, detail, stage=0)
data_process(train, test, detail, stage=1)

In [56]:
nlp_cols = ['房屋优势','核心卖点','户型介绍','周边配套','交通出行']

In [57]:
#3.对训练集train_test_split
X = train[['城市',
           '建筑结构',
           '装修情况',
           '配备电梯',
           '房屋年限',
           '产权所属',
           '室',
           '厅',
           '厨',
           '卫',
           '建筑面积',
           '套内面积',
           '梯户比值',
           'lon',
           'lat',
           'distance'
]]

y = train['log_prices']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
# 文本型变量

In [65]:
# 数值型变量
numerical_cols=['室', '厅', '厨', '卫', '建筑面积', '梯户比值', 'lon', 'lat', 'distance']
categorical_cols=['建筑结构', '装修情况', '配备电梯', '房屋年限', '产权所属']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OrdinalEncoder(), categorical_cols)
    ]
)



X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [ ]:
try:
    X_test_processed = preprocessor.transform(X_test)
except ValueError as e:
    print("Error occurred during transformation:")
    print(e)
    for col_index, col_name in enumerate(categorical_cols):
        train_categories = preprocessor.named_transformers_['cat'].categories_[col_index]
        test_categories = X_test[col_name].unique()
        unknown_categories = set(test_categories) - set(train_categories)

        if unknown_categories:
            print(f"Column '{col_name}' has unknown categories: {unknown_categories}")

In [ ]:
# 2.设置xgboost和elastic net的调参网格
xgb_param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2]
}
elastic_net_param_grid = {
    'alpha': [0.01, 0.1, 1.0],
    'l1_ratio': [0.1, 0.5, 0.9],  # l1:LASSO
}

# 3.对xgb做GridSearch
xgb_model=xgb.XGBRegressor(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_grid_search = GridSearchCV(xgb_model, xgb_param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
xgb_grid_search.fit(X_train_processed, y_train)
# 获取最佳的xgb模型
best_xgb_model = xgb_grid_search.best_estimator_

# 4.使用最佳xgb分类
xgb_train_pred = best_xgb_model.predict(X_train_processed)
xgb_test_pred = best_xgb_model.predict(X_test_processed)

# 5. 使用 XGBoost 预测结果作为新的特征，准备训练第二层模型（ElasticNet）
X_train_stack = pd.DataFrame(xgb_train_pred)
X_test_stack = pd.DataFrame(xgb_test_pred)

In [68]:
# 第二层:线性模型

# 1.对elastic net做GridSearch
elastic_net = make_pipeline(StandardScaler(), ElasticNet())

# 设置elastic net调参网格
elastic_net_param_grid = {
    'elasticnet__alpha': [0.1, 0.5, 1.0, 2.0],
    'elasticnet__l1_ratio': [0.1, 0.5, 0.9]
}
# make pipeline的作用是什么??
elastic_net_grid_search = GridSearchCV(elastic_net, elastic_net_param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
elastic_net_grid_search.fit(X_train_stack, y_train)
# 获取最佳EN模型
best_elastic_net_model = elastic_net_grid_search.best_estimator_

# 2.用最优的EN模型进行预测
elastic_net_pred = best_elastic_net_model.predict(X_test_stack)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


In [69]:
# 3.计算mean squared eror
mse = mean_squared_error(y_test, elastic_net_pred)
print(f'Mean Squared Error of the stacked model: {mse}')

Mean Squared Error of the stacked model: 0.025308055438806296


In [71]:
# 三、生成预测结果-xgb
X_input = test[[
            '城市',
           '建筑结构',
           '装修情况',
           '配备电梯',
           '房屋年限',
           '产权所属',
           '室',
           '厅',
           '厨',
           '卫',
           '建筑面积',
           '套内面积',
           '梯户比值',
           'lon',
           'lat',
           'distance'
]]
# xgb
X_input_processed = preprocessor.transform(X_input)

xgb_input_pred = best_xgb_model.predict(X_input_processed)
X_input_stack = pd.DataFrame(xgb_input_pred)
# EN
test['log_prices'] = best_elastic_net_model.predict(X_input_stack)
test['Price'] = np.exp(test['log_prices'])

# 生成csv
output = test[['ID','Price']]
output.to_csv('predictions.csv', index=False)